<a href="https://colab.research.google.com/github/vivek6311/Artificial-Intelligence-with-Python/blob/master/AU_Calibration_Gender_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [10]:
# Load the data
df = pd.read_excel('Calibrations_data.xlsx', sheet_name='Sheet1')
df.head()

,Interview number,Weight,Resp Weight,AWC By Longevity,Disp Format AU,Male,Female,Age_29,Age_30-44,Age_45+,...,VuseGo.3,Waka Klic.3,Waka Mini.3,Whoop Disposable Bar.3,WIIP Wiipstick.3,Wondervape 7000.3,X BAR.3,YOLO BAR.3,Ypsilos.3,Disp Other.3
0,1000006,0.946097,NaN,0.0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000040,1.042932,NaN,0.0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1000043,1.042932,NaN,0.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1000058,0.870052,NaN,0.0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000061,0.959103,NaN,2.8,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Remove the first row
#df = df.iloc[0:].reset_index(drop=True)

In [11]:
df.head()

,Interview number,Weight,Resp Weight,AWC By Longevity,Disp Format AU,Male,Female,Age_29,Age_30-44,Age_45+,...,VuseGo.3,Waka Klic.3,Waka Mini.3,Whoop Disposable Bar.3,WIIP Wiipstick.3,Wondervape 7000.3,X BAR.3,YOLO BAR.3,Ypsilos.3,Disp Other.3
0,1000006,0.946097,NaN,0.0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000040,1.042932,NaN,0.0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1000043,1.042932,NaN,0.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1000058,0.870052,NaN,0.0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000061,0.959103,NaN,2.8,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Extract the relevant columns
weights = df['Weight'].values
au_brand_columns = df.iloc[:, 22:222]  # Columns W to FZ (0-based index 22 to 221)
male, female = df['Male'].values, df['Female'].values
age_29, age_30_44, age_45_plus = df['Age_29'].values, df['Age_30-44'].values, df['Age_45+'].values

In [13]:
# Calculate initial demographic proportions
def calc_proportion(feature):
    return np.sum(feature * weights) / np.sum(weights)

initial_props = {feature: calc_proportion(df[feature].values) for feature in ['Male', 'Female', 'Age_29', 'Age_30-44', 'Age_45+']}

In [14]:
# Define the objective function to minimize
def objective_function(x):
    x_reshaped = x.reshape(au_brand_columns.shape)
    new_au_calib = np.sum(weights * x_reshaped) / np.sum(weights)
    target_diff = (new_au_calib - 0.60) ** 2  # Squared error for smooth optimization

    # Maintain demographic proportions
    demographic_diff = sum(
        (np.sum(df[feature].values * weights * x_reshaped) / np.sum(weights * x_reshaped) - initial_props[feature]) ** 2
        for feature in ['Male', 'Female', 'Age_29', 'Age_30-44', 'Age_45+']
    )

    return target_diff + demographic_diff

In [15]:
# Set bounds and initial guess
bounds = [(0, 1)] * au_brand_columns.size
initial_guess = au_brand_columns.values.flatten()

In [16]:
# Perform the optimization
result = minimize(objective_function, initial_guess, bounds=bounds, method='L-BFGS-B')

ValueError: operands could not be broadcast together with shapes (73,) (73,200) 

In [17]:
# Extract the relevant columns
weights = df['Weight'].values.reshape(-1, 1)  # Reshape to (73,1) for broadcasting
au_brand_columns = df.iloc[:, 22:222].values  # Columns W to FZ (0-based index 22 to 221)
male, female = df['Male'].values, df['Female'].values
age_29, age_30_44, age_45_plus = df['Age_29'].values, df['Age_30-44'].values, df['Age_45+'].values

In [18]:
# Calculate initial demographic proportions
def calc_proportion(feature):
    return np.sum(feature * weights.flatten()) / np.sum(weights)

initial_props = {feature: calc_proportion(df[feature].values) for feature in ['Male', 'Female', 'Age_29', 'Age_30-44', 'Age_45+']}


In [19]:
# Define the objective function to minimize
def objective_function(x):
    x_reshaped = x.reshape(au_brand_columns.shape)
    new_au_calib = np.sum(weights * x_reshaped) / np.sum(weights)
    target_diff = (new_au_calib - 0.60) ** 2  # Squared error for smooth optimization

    # Maintain demographic proportions
    demographic_diff = sum(
        (np.sum(df[feature].values * weights.flatten() * np.sum(x_reshaped, axis=1)) / np.sum(weights.flatten() * np.sum(x_reshaped, axis=1)) - initial_props[feature]) ** 2
        for feature in ['Male', 'Female', 'Age_29', 'Age_30-44', 'Age_45+']
    )

    return target_diff + demographic_diff

In [20]:
# Set bounds and initial guess
bounds = [(0, 1)] * au_brand_columns.size
initial_guess = au_brand_columns.flatten()

In [21]:
# Perform the optimization
result = minimize(objective_function, initial_guess, bounds=bounds, method='L-BFGS-B')

In [22]:
# Update the dataframe with optimized values
df.iloc[:, 22:222] = result.x.reshape(au_brand_columns.shape)

<ipython-input-22-f12933fe4723>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4.93128864e-04 6.00573185e-04 0.00000000e+00 2.21884028e-04
 5.52300113e-04 0.00000000e+00 5.43601154e-04 8.07613411e-04
 0.00000000e+00 2.41277553e-04 5.01020000e-04 0.00000000e+00
 4.99907464e-04 8.07613411e-04 0.00000000e+00 2.65972582e-04
 5.44810956e-04 0.00000000e+00 2.65972582e-04 7.32627947e-04
 0.00000000e+00 4.99907464e-04 6.00573185e-04 0.00000000e+00
 2.41277553e-04 5.52300113e-04 0.00000000e+00 4.99907493e-04
 8.07613411e-04 0.00000000e+00 2.44594166e-04 6.00573185e-04
 0.00000000e+00 5.43601154e-04 8.07613411e-04 0.00000000e+00
 2.21884028e-04 5.44810956e-04 0.00000000e+00 2.82570582e-04
 8.58012179e-04 0.00000000e+00 4.53491975e-04 5.44810956e-04
 0.00000000e+00 2.21884028e-04 6.00573185e-04 0.00000000e+00
 1.00000000e+00 9.33005432e-04 0.00000000e+00 2.44594166e-04
 5.44810956e-04 0.00000000e+00 5.43601154e-04 8.782017

In [23]:
# Save the calibrated data
df.to_excel('Calibrated_Data.xlsx', index=False)